# Sample SDK

## Pandas HTML + BeautifulSoup



In [1]:
!pip install requests-random-user-agent
!pip install pandas
!pip install bs4
!pip install lxml

In [2]:
import multiprocessing
multiprocessing.cpu_count()

6

In [3]:

import requests
import requests_random_user_agent
#s = requests.Session()
#print(s.headers['User-Agent'])

# Without a session
resp = requests.get('https://httpbin.org/user-agent')
print(resp.json()['user-agent'])

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36


In [4]:
import concurrent.futures
import requests
import requests_random_user_agent
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

def _download_SDK(url):

  df_tempSDK = pd.DataFrame()

  rq = requests.get(url)
  
  if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
    exit

  main_data = rq.text

  # dataset from table
  df_tempSDK = pd.read_html(main_data, index_col=0)[0]

  main_soup = BeautifulSoup(main_data, 'html.parser')
  
  main_names = main_soup.find_all('tr')[1:245]

  list_urlSDK = []

  # Head url for meta_url
  head_Url= 'https://www.programmableweb.com'

  for row in main_names:
       text = row.find_all('td')[0]      
       list_urlSDK.append( (head_Url + str(text).partition('<a href="')[2].partition('">')[0])) # Meta URL 


  df_tempSDK['Meta_Url'] = list_urlSDK

  return df_tempSDK


def download_SDK(bulk_urls, num_Splits):
  df_temp = pd.DataFrame()

  # Split urls
  urls_splited  = np.array_split(bulk_urls, num_Splits) # max workers

  tasks = []

  for split in range(len(urls_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(urls_splited)) as executor:
      for url in urls_splited[split]:    
        tasks.append(executor.submit(_download_SDK, url))
       ## mb call with bulk urls instead of one by one
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp


In [5]:
SDK_urls=[]
for i in range(776): ## web pages?.. 775+1
    main_url = 'https://www.programmableweb.com/category/all/sdk&page=' + str(i) ## parametrizar + comprobar que hay datos
    SDK_urls.append(main_url)

In [6]:
#SDK_urls
df_SDK = pd.DataFrame()
df_SDK = download_SDK(SDK_urls, 10) # Num workers
df_SDK.drop('Submitted', inplace=True, axis=1)
df_SDK

,Related APIs,Category,Meta_Url
SDK Name,,,
Adobe PDF Embed JavaScript SDK,Adobe PDF Embed,PDF,https://www.programmableweb.com/sdk/adobe-pdf-...
WaifuAI Chatbot NodeJS SDK,WaifuAI,Chat,https://www.programmableweb.com/sdk/waifuai-ch...
SecuGen Web JavaScript SDK,SecuGen Web,Biometrics,https://www.programmableweb.com/sdk/secugen-we...
Neuro AI SDK,Neuro AI,Machine Learning,https://www.programmableweb.com/sdk/neuro-ai-sdk
CurrencyAPI NodeJS SDK,CurrencyAPI,Currency,https://www.programmableweb.com/sdk/currencyap...
CurrencyAPI PHP SDK,CurrencyAPI,Currency,https://www.programmableweb.com/sdk/currencyap...
mypromo PHP SDK,mypromo,Printing,https://www.programmableweb.com/sdk/mypromo-ph...
Shotstack Ruby SDK,Shotstack,Video,https://www.programmableweb.com/sdk/shotstack-...
Shotstack PHP SDK,Shotstack,Video,https://www.programmableweb.com/sdk/shotstack-...


## Meta URL Processing

In [7]:
# Creates new columns
df_SDK['Description'] = ""
df_SDK['Languages'] = ""
df_SDK['Provider'] = ""
df_SDK['URL'] = ""
df_SDK['Repository'] = ""

In [8]:
def _download_meta_url(df_source):

  for i in range(len(df_source)):

    meta_url = df_source['Meta_Url'][i]
    rq = requests.get(meta_url)

    if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
      continue

    meta_data = rq.text

    meta_soup = BeautifulSoup(meta_data, 'html.parser')

    # Update Description from the meta url
    meta_description = str(meta_soup.find('div', class_='tabs-header_description')).partition('">')[2].partition('</')[0]

    #print(meta_description)
    df_source['Description'][i] = meta_description 

    meta_specs = meta_soup.find('div', class_='section specs')

    #print(meta_specs)

    for lab in meta_specs.select("label"):   

      # Search for Related APIs
      if (lab.text.lower().find("related apis") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Related APIs'][i] =   lab.find_next_sibling().text

      # Search for Languages
      if (lab.text.lower().find("languages") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Languages'][i] =   lab.find_next_sibling().text

       # Search for Categories and remplace them
      if (lab.text.lower().find("categories") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Category'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("sdk provider") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Provider'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("url") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['URL'][i] =   lab.find_next_sibling().text

      # Search for Provider
      if (lab.text.lower().find("repository") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Repository'][i] =   lab.find_next_sibling().text

    #print(df_source)

  return df_source

def download_meta_url(df_sourceCode, num_Splits):
  # Split dataframe
  dtframe_splited  = np.array_split(df_sourceCode, num_Splits) # max workers == num_Splits
  tasks = []

  for split in range(len(dtframe_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(dtframe_splited)) as executor:
      # Download dataframes splited
      tasks.append(executor.submit(_download_meta_url, dtframe_splited[split]))

  df_temp = pd.DataFrame()
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp


In [9]:
df_SDK_metaData = pd.DataFrame()
df_SDK_metaData = download_meta_url(df_SDK, 10)
df_SDK_metaData

,Related APIs,Category,Meta_Url,Description,Languages,Provider,URL,Repository
SDK Name,,,,,,,,
Adobe PDF Embed JavaScript SDK,Adobe PDF Embed Indirect API v2.15.0,"PDF, Documents",https://www.programmableweb.com/sdk/adobe-pdf-...,The PDF Embed Javascript SDK lets developers ...,JavaScript,Adobe,https://www.adobe.io/apis/documentcloud/dcsdk/...,
WaifuAI Chatbot NodeJS SDK,WaifuAI Indirect API v1,"Chat, Artificial Intelligence, Avatars, Bots",https://www.programmableweb.com/sdk/waifuai-ch...,Waifus are anime-themed personal assistants. ...,Node.js,WaifuAI,https://waifuai.com/,
SecuGen Web JavaScript SDK,SecuGen Web Indirect API v1,Biometrics,https://www.programmableweb.com/sdk/secugen-we...,The service offers easy to integrate fingerpr...,JavaScript,SecuGen Corporation,https://secugen.com/products/webapi/,
Neuro AI SDK,Neuro AI Indirect API v1,Machine Learning,https://www.programmableweb.com/sdk/neuro-ai-sdk,Neuro AI provides infrastructure for machine ...,Python,Neuro Ai,https://www.getneuro.ai/about,
CurrencyAPI NodeJS SDK,CurrencyAPI REST API v1,"Currency, Accounts, Data, Financial, Invoicing...",https://www.programmableweb.com/sdk/currencyap...,NodeJs wrapper for CurrencyApi.net endpoints....,Node.js,CurrencyAPI,https://github.com/houseofapis/currencyapi-node,https://github.com/houseofapis/currencyapi-node
CurrencyAPI PHP SDK,CurrencyAPI REST API v1,"Currency, Accounts, Financial, Invoicing, Mone...",https://www.programmableweb.com/sdk/currencyap...,PHP wrapper for CurrencyApi.net endpoints.\nP...,PHP,CurrencyAPI,https://github.com/houseofapis/currencyapi-php,https://github.com/houseofapis/currencyapi-php
mypromo PHP SDK,mypromo REST API v1,"Printing, Customization, eCommerce, Products",https://www.programmableweb.com/sdk/mypromo-ph...,mypromo PHP SDK simplifies the use of API for...,PHP,mypromo,https://docs.api.mypromo.com,https://github.com/MyPromoConnect/connect-sdk
Shotstack Ruby SDK,Shotstack REST API v1,"Video, Audio, Images, Media",https://www.programmableweb.com/sdk/shotstack-...,This is the Ruby SDK for interacting with the...,Ruby,Shotstack,https://shotstack.io/docs/api/,https://github.com/shotstack/shotstack-sdk-ruby
Shotstack PHP SDK,Shotstack REST API v1,"Video, Audio, Images, Media",https://www.programmableweb.com/sdk/shotstack-...,This is the PHP SDK for interacting with the ...,PHP,Shotstack,https://shotstack.io/docs/api/#shotstack,https://github.com/shotstack/shotstack-sdk-php


In [10]:
# Meta_Url could be used to check for updates on the source website. That uses only +600 web requests instead of +15k
# save a copy of the original dataframe to check for updates based on the meta url or other fields
df_SDK_metaData.reset_index(inplace=True)
df_export_SDK = df_SDK_metaData.copy()
df_SDK_metaData.drop('Meta_Url', inplace=True, axis=1)

df_SDK_metaData

,SDK Name,Related APIs,Category,Description,Languages,Provider,URL,Repository
0,Adobe PDF Embed JavaScript SDK,Adobe PDF Embed Indirect API v2.15.0,"PDF, Documents",The PDF Embed Javascript SDK lets developers ...,JavaScript,Adobe,https://www.adobe.io/apis/documentcloud/dcsdk/...,
1,WaifuAI Chatbot NodeJS SDK,WaifuAI Indirect API v1,"Chat, Artificial Intelligence, Avatars, Bots",Waifus are anime-themed personal assistants. ...,Node.js,WaifuAI,https://waifuai.com/,
2,SecuGen Web JavaScript SDK,SecuGen Web Indirect API v1,Biometrics,The service offers easy to integrate fingerpr...,JavaScript,SecuGen Corporation,https://secugen.com/products/webapi/,
3,Neuro AI SDK,Neuro AI Indirect API v1,Machine Learning,Neuro AI provides infrastructure for machine ...,Python,Neuro Ai,https://www.getneuro.ai/about,
4,CurrencyAPI NodeJS SDK,CurrencyAPI REST API v1,"Currency, Accounts, Data, Financial, Invoicing...",NodeJs wrapper for CurrencyApi.net endpoints....,Node.js,CurrencyAPI,https://github.com/houseofapis/currencyapi-node,https://github.com/houseofapis/currencyapi-node
5,CurrencyAPI PHP SDK,CurrencyAPI REST API v1,"Currency, Accounts, Financial, Invoicing, Mone...",PHP wrapper for CurrencyApi.net endpoints.\nP...,PHP,CurrencyAPI,https://github.com/houseofapis/currencyapi-php,https://github.com/houseofapis/currencyapi-php
6,mypromo PHP SDK,mypromo REST API v1,"Printing, Customization, eCommerce, Products",mypromo PHP SDK simplifies the use of API for...,PHP,mypromo,https://docs.api.mypromo.com,https://github.com/MyPromoConnect/connect-sdk
7,Shotstack Ruby SDK,Shotstack REST API v1,"Video, Audio, Images, Media",This is the Ruby SDK for interacting with the...,Ruby,Shotstack,https://shotstack.io/docs/api/,https://github.com/shotstack/shotstack-sdk-ruby
8,Shotstack PHP SDK,Shotstack REST API v1,"Video, Audio, Images, Media",This is the PHP SDK for interacting with the ...,PHP,Shotstack,https://shotstack.io/docs/api/#shotstack,https://github.com/shotstack/shotstack-sdk-php
9,Shotstack NodeJS SDK,Shotstack REST API v1,"Video, Audio, Images, Media",This is the Node.JS SDK for interacting with ...,Node.js,Shotstack,https://shotstack.io/docs/api/#shotstack,https://github.com/shotstack/shotstack-sdk-node


In [13]:
from datetime import datetime
datetime = datetime.now()

## Export the original - Meta_Url
# To CSV (index True 0,1,2...)
df_export_SDK.to_csv(r'C:/Users/David/Desktop/DataFrames/SDK_' + datetime.now().strftime('%d_%m_%Y') + '.csv', index = True, header = True)

# To JSON (columns format index True 0,1,2...)
df_export_SDK.to_json(r'C:/Users/David/Desktop/DataFrames/SDK_' + datetime.now().strftime('%d_%m_%Y') + '.json')


## Export the original + Meta_Url

# To CSV (index True 0,1,2...)
df_SDK_metaData.to_csv(r'C:/Users/David/Desktop/DataFrames/MetaUrl_SDK_' + datetime.now().strftime('%d_%m_%Y') + '.csv', index = True, header = True)

# To JSON (columns format index True 0,1,2...)
df_SDK_metaData.to_json(r'C:/Users/David/Desktop/DataFrames/MetaUrl_SDK_' + datetime.now().strftime('%d_%m_%Y') + '.json')